In [114]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark

# install pyspark
!pip3 install pyspark==3.2.0

# install graphframes
!pip3 install graphframes

In [115]:
!pip show pyspark

Name: pyspark
Version: 3.2.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: py4j
Required-by: 


In [116]:
#import the packages
from pyspark import *
from pyspark.sql import *
from graphframes import *
import findspark
import pandas as pd
from pyspark.sql.functions import mean as sqlmean


findspark.init()

# Start a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

KeyError: ignored

In [104]:
import psycopg2

In [105]:
# access the postgresql server
conn = psycopg2.connect(
    host="codd04.research.northwestern.edu",
    port = "5433",
    database="postgres",
    user="cpdbstudent",
    password="DataSci4AI")

In [106]:
cursor = conn.cursor()

In [107]:
edges_query = "SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship FROM data_officerallegation da1 JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC;"

In [108]:
nodes_query = "SELECT  id, first_name || ' ' || last_name officer_name, allegation_count, civilian_allegation_percentile FROM data_officer;"

In [109]:
cursor.execute(edges_query)
edges = cursor.fetchall()

df_edges = pd.DataFrame(edges)
colnames = [desc[0] for desc in cursor.description]
df_edges.columns = colnames


In [110]:
cursor.execute(nodes_query)
nodes = cursor.fetchall()

df_nodes = pd.DataFrame(nodes)
colnames = [desc[0] for desc in cursor.description]
df_nodes.columns = colnames


In [117]:
edges = spark.createDataFrame(df_edges)

Py4JError: ignored

In [ ]:
nodes = spark.createDataFrame(df_nodes)

nodes.describe().show()

In [ ]:
cpdb = GraphFrame(nodes, edges)

In [113]:
from pyspark.sql.functions import corr
tc_cpdb = cpdb.triangleCount()
tc_cpdb.select("civilian_allegation_percentile").describe().show()
tc_cpdb.select("id", "count", "civilian_allegation_percentile").corr("count", "civilian_allegation_percentile")

Py4JError: ignored

In [112]:
lp_cpdb = cpdb.labelPropagation(maxIter=15)
lp_cpdb.select("id","label", "allegation_count", "civilian_allegation_percentile").sort(["label"], ascending=False).show()

KeyError: ignored

In [ ]:
pr_cpdb = cpdb.pageRank(resetProbability=0.15, tol=0.01)
pr_cpdb.vertices.orderBy('pagerank', ascending=False).show()
pr_cpdb.vertices.orderBy('pagerank', ascending=False).select("pagerank").describe().show()
pr_cpdb.vertices.select("id","pagerank", "allegation_count", "civilian_allegation_percentile").corr("pagerank", "civilian_allegation_percentile")

In [ ]:
from graphframes.lib import AggregateMessages as AM


msgToSrc = AM.dst["civilian_allegation_percentile"]
msgToDst = AM.src["civilian_allegation_percentile"]
agg = cpdb.aggregateMessages(
    sqlmean(AM.msg).alias("meanCivilianAllegationPercent"),
    sendToSrc=msgToSrc,
    sendToDst=msgToDst)

agg.sort(['meanCivilianAllegationPercent'],ascending=False).show()
agg.select('meanCivilianAllegationPercent').describe().show()